# Variable dependiente: Acceso a Educación Universitaria (dicotómica)

## 1. Cargar data

In [1]:
import pandas as  pd, numpy as np
import variables as vb

In [2]:
# Cargar datos
path = r'..\..\output\data_preprocess\dfs_0_uni.csv'
data_original = pd.read_csv( path )

In [3]:
data_original.shape

(875, 313)

## 2. Scale only numeric vars

In [4]:
# https://stackoverflow.com/questions/38420847/apply-standardscaler-to-parts-of-a-data-set

from sklearn.preprocessing import StandardScaler

data = data_original.copy()

numeric_vars = [col for col in data.columns if col in vb.num_vars ]

cols = data[ numeric_vars ]
scaler = StandardScaler().fit( cols.values )
cols = scaler.transform( cols.values )

data[ numeric_vars ] = cols

## 3. Split variables

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
dep_var = [ 'e_educacion_uni' ]
pred_vars = [col for col in data.columns if col not in vb.dep_vars and col not in dep_var ]
x_train, x_test, y_train, y_test = train_test_split( data[ pred_vars ], data[ 'e_educacion_uni' ], test_size = 0.25 )

## 4. Logistic Regression

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, log_loss, roc_auc_score, f1_score, classification_report

In [8]:
%%time

# Implementing the model
lg_model = LogisticRegression( max_iter = 10000 ).fit( x_train, y_train )

# Predict over test set
y_lg_pred_class = lg_model.predict( x_test )
y_lg_pred_prob = lg_model.predict_proba( x_test )[ :, 1 ]

Wall time: 241 ms


In [9]:
# Calculating metrics
columns = [ 'no', 'uni' ]
lg_report = classification_report(y_test, y_lg_pred_class, target_names = columns, output_dict = True )

lg_no_precision = lg_report[ 'no' ][ 'precision' ]
lg_no_recall = lg_report[ 'no' ][ 'recall' ]
lg_no_f1_score = lg_report[ 'no' ][ 'f1-score' ]

lg_uni_precision = lg_report[ 'uni' ][ 'precision' ]
lg_uni_recall = lg_report[ 'uni' ][ 'recall' ]
lg_uni_f1_score = lg_report[ 'uni' ][ 'f1-score' ]

accuracy_lg = accuracy_score( y_test, y_lg_pred_class )
log_loss_lg = log_loss( y_test, y_lg_pred_class )
roc_auc_lg = roc_auc_score( y_test, y_lg_pred_prob )

Info:
* prod_proba according to https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html

## 5. Regularization Methods (Lasso, Ridge and Elastic Net)

In [10]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import GridSearchCV

## 5.1. Lasso

In [11]:
%%time

# Implementing the model
lasso_model = LogisticRegressionCV( penalty = 'l1', solver = 'saga', cv = 10, random_state = 0, max_iter = 10000 ).\
                               fit( x_train, y_train )

# Predict over test set
y_lasso_pred_class = lasso_model.predict( x_test )
y_lasso_pred_prob = lasso_model.predict_proba( x_test )[ :, 1 ]

Wall time: 5min 20s


In [12]:
# Calculating metrics
columns = [ 'no', 'uni' ]
lasso_report = classification_report(y_test, y_lasso_pred_class, target_names = columns, output_dict = True )

lasso_no_precision = lasso_report[ 'no' ][ 'precision' ]
lasso_no_recall = lasso_report[ 'no' ][ 'recall' ]
lasso_no_f1_score = lasso_report[ 'no' ][ 'f1-score' ]

lasso_uni_precision = lasso_report[ 'uni' ][ 'precision' ]
lasso_uni_recall = lasso_report[ 'uni' ][ 'recall' ]
lasso_uni_f1_score = lasso_report[ 'uni' ][ 'f1-score' ]

accuracy_lasso = accuracy_score( y_test, y_lasso_pred_class )
log_loss_lasso = log_loss( y_test, y_lasso_pred_class )
roc_auc_lasso = roc_auc_score( y_test, y_lasso_pred_prob )

## 5.2. Ridge

In [13]:
%%time

# Implementing the model
ridge_model = LogisticRegressionCV( penalty = 'l2', solver = 'saga', cv = 10, random_state = 0, max_iter = 10000 ).\
                               fit( x_train, y_train )

# Predict over test set
y_ridge_pred_class = ridge_model.predict( x_test )
y_ridge_pred_prob = ridge_model.predict_proba( x_test )[ :, 1 ]

Wall time: 3min 45s


In [14]:
# Calculating metrics
columns = [ 'no', 'uni' ]
ridge_report = classification_report( y_test, y_ridge_pred_class, target_names = columns, output_dict = True )

ridge_no_precision = ridge_report[ 'no' ][ 'precision' ]
ridge_no_recall = ridge_report[ 'no' ][ 'recall' ]
ridge_no_f1_score = ridge_report[ 'no' ][ 'f1-score' ]

ridge_uni_precision = ridge_report[ 'uni' ][ 'precision' ]
ridge_uni_recall = ridge_report[ 'uni' ][ 'recall' ]
ridge_uni_f1_score = ridge_report[ 'uni' ][ 'f1-score' ]

accuracy_ridge = accuracy_score( y_test, y_ridge_pred_class )
log_loss_ridge = log_loss( y_test, y_ridge_pred_class )
roc_auc_ridge = roc_auc_score( y_test, y_ridge_pred_prob )

### 5.3. Elastic Net

In [15]:
%%time

# Implementing the model
elasticnet_model = LogisticRegressionCV( penalty = 'elasticnet', solver = 'saga', cv = 10, random_state = 0, l1_ratios = [ 0.5 ], max_iter = 10000 ).\
                                    fit( x_train, y_train )

# Predict over test set
y_elasticnet_pred_class = elasticnet_model.predict( x_test )
y_elasticnet_pred_prob = elasticnet_model.predict_proba( x_test )[ :, 1 ]

Wall time: 5min 28s


In [16]:
# Calculating metrics
columns = [ 'no', 'uni' ]
elasticnet_report = classification_report( y_test, y_elasticnet_pred_class, target_names = columns, output_dict = True )

elasticnet_no_precision = elasticnet_report[ 'no' ][ 'precision' ]
elasticnet_no_recall = elasticnet_report[ 'no' ][ 'recall' ]
elasticnet_no_f1_score = elasticnet_report[ 'no' ][ 'f1-score' ]

elasticnet_uni_precision = elasticnet_report[ 'uni' ][ 'precision' ]
elasticnet_uni_recall = elasticnet_report[ 'uni' ][ 'recall' ]
elasticnet_uni_f1_score = elasticnet_report[ 'uni' ][ 'f1-score' ]

accuracy_elasticnet = accuracy_score( y_test, y_elasticnet_pred_class )
log_loss_elasticnet = log_loss( y_test, y_elasticnet_pred_class )
roc_auc_elasticnet = roc_auc_score( y_test, y_elasticnet_pred_prob )

C:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## 6. Random Forest

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [28]:
%%time

# Set the model
rf_model = RandomForestClassifier( random_state = 0 )

# Define param grid
rf_param_grid = {
    'n_estimators': [ 500, 1000, 2000 ],
    'max_features': [ 'auto', 'sqrt', 'log2' ]
}

# Grid search
rf_search = GridSearchCV( estimator = rf_model,
                          param_grid = rf_param_grid )

# Fit to data
rf_search.fit( x_train, y_train )

# Print best params and best score
print( rf_search.best_params_ )

# Select best params
rf_max_features = rf_search.best_params_[ 'max_features' ] 
rf_n_estimators = rf_search.best_params_[ 'n_estimators' ] 

{'max_features': 'log2', 'n_estimators': 2000}
Wall time: 1min 57s


In [29]:
# Train the optimal model
rf_optimal_model = RandomForestClassifier( max_features = rf_max_features, 
                                           n_estimators = rf_n_estimators )
rf_optimal_model.fit( x_train, y_train )

# Apply over test set
y_rf_pred_class = rf_optimal_model.predict( x_test )
y_rf_pred_prob = rf_optimal_model.predict_proba( x_test )[ :, 1 ]

In [30]:
# Calculating metrics
columns = [ 'no', 'uni' ]
rf_report = classification_report(y_test, y_rf_pred_class, target_names = columns, output_dict = True )

rf_no_precision = rf_report[ 'no' ][ 'precision' ]
rf_no_recall = rf_report[ 'no' ][ 'recall' ]
rf_no_f1_score = rf_report[ 'no' ][ 'f1-score' ]

rf_uni_precision = rf_report[ 'uni' ][ 'precision' ]
rf_uni_recall = rf_report[ 'uni' ][ 'recall' ]
rf_uni_f1_score = rf_report[ 'uni' ][ 'f1-score' ]

accuracy_rf = accuracy_score( y_test, y_rf_pred_class )
log_loss_rf = log_loss( y_test, y_rf_pred_class )
roc_auc_rf = roc_auc_score( y_test, y_rf_pred_prob )

## 7. Boosted Trees

In [21]:
from xgboost import XGBClassifier

C:\Anaconda\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [32]:
%%time

# Set the model
xgb_model = XGBClassifier( use_label_encoder = False, objective = 'binary:logistic', verbosity = 0 )

# Define param grid
xgb_param_grid = {
    'n_estimators': [ 500, 1000, 2000 ],
    'learning_rate': [0.1, 0.5, 1]
}

# Grid search
xgb_search = GridSearchCV( estimator = xgb_model,
                           param_grid = xgb_param_grid )

# Fit to data
xgb_search.fit( x_train, y_train )

# Print best params and best score
print( xgb_search.best_params_ )

# Select best params
xgb_learning_rate = xgb_search.best_params_[ 'learning_rate' ] 
xgb_n_estimators = xgb_search.best_params_[ 'n_estimators' ] 

C:\Anaconda\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Anaconda\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Anaconda\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Anaconda\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtyp

{'learning_rate': 0.1, 'n_estimators': 500}
Wall time: 2min 23s


In [33]:
# Train the optimal model
xgb_optimal_model = XGBClassifier( objective = 'binary:logistic', 
                                   verbosity = 0,
                                   learning_rate = xgb_learning_rate, 
                                   n_estimators = xgb_n_estimators )
xgb_optimal_model.fit( x_train, y_train )

# Apply over test set
y_xgb_pred_class = xgb_optimal_model.predict( x_test )
y_xgb_pred_prob = xgb_optimal_model.predict_proba( x_test )[ :, 1 ]

C:\Anaconda\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [34]:
# Calculating metrics
columns = [ 'no', 'uni' ]
xgb_report = classification_report(y_test, y_xgb_pred_class, target_names = columns, output_dict = True )

xgb_no_precision = xgb_report[ 'no' ][ 'precision' ]
xgb_no_recall = xgb_report[ 'no' ][ 'recall' ]
xgb_no_f1_score = xgb_report[ 'no' ][ 'f1-score' ]

xgb_uni_precision = xgb_report[ 'uni' ][ 'precision' ]
xgb_uni_recall = xgb_report[ 'uni' ][ 'recall' ]
xgb_uni_f1_score = xgb_report[ 'uni' ][ 'f1-score' ]

accuracy_xgb = accuracy_score( y_test, y_xgb_pred_class )
log_loss_xgb = log_loss( y_test, y_xgb_pred_class )
roc_auc_xgb = roc_auc_score( y_test, y_xgb_pred_prob )

## 7. Resultados

In [35]:
table = np.zeros( ( 6, 8 ) )

table[ 0 ] = [ accuracy_lg, roc_auc_lg, lg_no_precision, lg_no_recall, 
               lg_no_f1_score, lg_uni_precision, lg_uni_recall, lg_uni_f1_score ]

table[ 1 ] = [ accuracy_lasso, roc_auc_lasso, lasso_no_precision, lasso_no_recall, 
               lasso_no_f1_score, lasso_uni_precision, lasso_uni_recall, lasso_uni_f1_score ]

table[ 2 ] = [ accuracy_ridge, roc_auc_ridge, ridge_no_precision, ridge_no_recall, 
               ridge_no_f1_score, ridge_uni_precision, ridge_uni_recall, ridge_uni_f1_score ]

table[ 3 ] = [ accuracy_elasticnet, roc_auc_elasticnet, elasticnet_no_precision, elasticnet_no_recall, 
               elasticnet_no_f1_score, elasticnet_uni_precision, elasticnet_uni_recall, elasticnet_uni_f1_score ]

table[ 4 ] = [ accuracy_rf, roc_auc_rf, rf_no_precision, rf_no_recall, 
               rf_no_f1_score, rf_uni_precision, rf_uni_recall, rf_uni_f1_score ]

table[ 5 ] = [ accuracy_xgb, roc_auc_xgb, xgb_no_precision, xgb_no_recall, 
               xgb_no_f1_score, xgb_uni_precision, xgb_uni_recall, xgb_uni_f1_score ]

colnames_table = [ "Overall_Accuracy", "Roc_Auc", "Ninguna_Precision", "Ninguna_Recall",
                   "Ninguna_F1_Score", "Uni_Precision", "Uni_Recall", "Uni_F1_Score" ]
                  
rownames_table = [ "Logistic Regression", "Lasso",
                   "Ridge", "Elastic Net",
                   "Random Forest", "Boosted Trees" ]

table_pandas = pd.DataFrame( table, columns = colnames_table )
table_pandas.index = rownames_table

table_pandas = table_pandas.round(3)
table_pandas

,Overall_Accuracy,Roc_Auc,Ninguna_Precision,Ninguna_Recall,Ninguna_F1_Score,Uni_Precision,Uni_Recall,Uni_F1_Score
Logistic Regression,0.658,0.651,0.750,0.765,0.757,0.429,0.409,0.419
Lasso,0.753,0.759,0.739,1.000,0.850,1.000,0.182,0.308
Ridge,0.721,0.779,0.715,1.000,0.834,1.000,0.076,0.141
Elastic Net,0.699,0.500,0.699,1.000,0.823,0.000,0.000,0.000
Random Forest,0.753,0.796,0.746,0.980,0.847,0.833,0.227,0.357
Boosted Trees,0.744,0.752,0.759,0.928,0.835,0.656,0.318,0.429


## 8. Feature map

In [26]:
# Random Forest
fp_randomforest = pd.Series( rf_optimal_model.feature_importances_, index = pred_vars).\
                  sort_values( ascending = False )
fp_randomforest.head(10)

c_gru71hd_     0.022865
e_p311t1_      0.022523
e_p311b_7_     0.020804
c_inghog2d_    0.019426
e_p311b_1_     0.017744
c_gru31hd_     0.017728
c_gru61hd_     0.017720
c_gru81hd_     0.017582
c_gru41hd_     0.017366
c_gru11hd_     0.015970
dtype: float64

In [27]:
# Boosted Trees
fp_xgboost = pd.Series( xgb_optimal_model.feature_importances_, index = pred_vars).\
           sort_values( ascending = False )
fp_xgboost.head(10)

e_p311a5_5_    0.043569
e_p414_09_     0.031993
e_p314b_1_     0.029848
e_p314b1_2_    0.027486
e_p4032_       0.024943
e_p311a5_2_    0.022030
e_p4095_       0.020767
e_p310_        0.019852
e_p4093_       0.018624
e_p314d_       0.017988
dtype: float32